In [ ]:
import os
import numpy as np
import pandas as pd
import torch as th
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/MyDrive/BCI Projects/EEG data from hands movement')

Mounted at /content/gdrive


In [ ]:
device = th.device('cuda' if th.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
DATA_PATH = os.path.join(os.getcwd(), 'data')

In [ ]:
users = [pd.read_csv(os.path.join(DATA_PATH, 'user_{}.csv'.format(x))) for x in ['a', 'b', 'c', 'd']]
users[0].head(10)

,Class,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,...,F8 beta std,F8 beta m,AF4 delta std,AF4 delta m,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m
0,1.0,3569.164550,2063.892754,1.673726,4.444736,0.526209,3.002088,1.425022,3.302739,3563.803888,...,45.468326,72.508750,3701.186330,2182.676835,18.192418,41.349662,16.004756,42.046467,46.280843,73.565719
1,1.0,3568.423670,2063.099248,1.897790,3.728823,1.304186,1.854353,1.366575,2.546458,3563.560922,...,36.551948,66.931186,3725.210509,2180.197439,8.820788,38.012788,19.601233,29.431054,38.559351,67.470041
2,1.0,3568.157929,2062.445859,2.798014,2.574504,1.120537,1.958819,0.982433,2.258622,3563.279981,...,40.754308,66.816547,3724.417296,2176.823208,18.159202,23.612639,14.378291,19.555084,43.210004,67.781924
3,1.0,3567.710021,2062.112673,2.181775,3.610507,0.629608,2.155876,0.856275,2.233711,3562.787801,...,38.074628,63.915386,3725.822160,2177.089059,19.737616,29.484396,15.793034,25.713513,39.250246,65.031031
4,1.0,3565.546124,2063.128867,1.685161,3.384311,0.677526,1.795798,0.927924,1.909810,3562.655091,...,35.357384,64.534645,3723.053978,2167.798335,8.429414,26.374975,14.920736,35.675266,33.901687,66.956313
5,1.0,3565.596693,2062.885545,1.138040,3.350574,0.514999,0.989174,1.500933,2.235967,3563.515323,...,37.827296,64.442073,3708.683926,2168.220614,13.144245,23.085914,8.461441,32.990127,38.525599,66.176828
6,1.0,3564.800801,2062.233256,1.538613,2.737740,0.541041,1.791602,1.351195,2.154442,3562.540480,...,42.671846,54.348776,3691.501051,2159.153230,10.686252,20.352354,9.454324,33.322877,44.028983,56.520105
7,1.0,3563.730766,2061.187840,0.566341,1.845693,1.501889,2.675382,1.048950,2.278802,3562.301877,...,30.369494,60.133904,3692.920771,2158.094932,5.863502,15.494530,17.407820,38.229668,30.766876,61.360470
8,1.0,3564.126792,2061.353290,0.648949,3.281405,1.054596,2.530217,1.127597,2.454784,3562.749854,...,34.359230,57.699328,3699.647985,2168.274449,4.754624,26.985426,20.919154,36.441315,35.586791,58.077899
9,1.0,3561.478553,2059.383202,0.624158,3.588017,1.036633,2.159542,1.298398,2.179527,3561.410591,...,19.224333,47.211981,3727.187011,2184.864177,12.083867,27.350869,15.128274,34.809569,21.248352,47.542259


In [ ]:
def data(users: list) -> tuple:
    df = pd.DataFrame(columns=users[0].columns)
    df['User'] = []

    for i, d in enumerate(users):
        d['User'] = [i]*len(d.Class)
        df = df.append(d, ignore_index=True)

    df = df.sample(frac=1.0, random_state=0).reset_index(drop=True)
    dummies = pd.get_dummies(df['Class'], prefix='Class')
    df = pd.concat([df, dummies], axis=1)
    df = df.drop('Class', axis=1)

    y = df['User']
    #y = pd.get_dummies(df['User'], prefix='User')
    X = df.drop('User', axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)

    scaler = StandardScaler()
    scaler.fit(X_train)

    return scaler.transform(X_train), scaler.transform(X_test), y_train.values, y_test.values

In [ ]:
X_train, X_test, y_train, y_test = data(users)

In [ ]:
class Data(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
            return len(self.y)
    def __getitem__(self, idx):
            return self.X[idx], self.y[idx]

In [ ]:
train_data = Data(X_train, y_train)
test_data = Data(X_test, y_test)

In [ ]:
train_data[5]

(array([ 0.21956886,  0.51115094,  0.58345047,  0.41821792, -0.05498414,
        -0.24504227,  0.4456752 ,  1.16485683,  1.01962325,  0.79844044,
        -0.57112277,  1.96598985, -0.29454074,  0.39013917,  0.95570086,
         1.45768955,  3.1534918 ,  3.59989975,  1.16528031,  0.78860397,
        -0.51824091,  0.2723731 , -0.42677837, -0.2281324 ,  1.31931857,
         1.95682733,  0.95102728,  0.7938862 ,  1.29961585,  1.31671411,
        -0.24973414,  0.17524624,  0.97395751,  1.2000712 ,  0.49819039,
         0.09044479,  0.71873319, -0.01191227, -0.26962269,  0.25563111,
         1.50670285,  2.04695744, -0.02342221, -0.21495064, -0.52596437,
        -0.47430536, -0.38535423, -0.38629046,  0.79338868,  1.2314558 ,
        -0.28056397,  0.98380708, -0.80846413,  0.50529097, -0.66874361,
        -0.60445327, -0.1734179 , -0.237219  , -0.19466168, -0.24459871,
        -0.26647197, -0.30497836, -0.30091735, -0.33562075, -0.33724168,
        -0.34315364, -0.24677484, -0.32568978, -0.3

In [ ]:
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=True)

In [ ]:
class UserPredictionModel(nn.Module):
    def __init__(self, input_size):
        super(UserPredictionModel, self).__init__()
        self.input_size = input_size

        self.model = nn.Sequential(
            nn.Linear(in_features=self.input_size, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=4),
            nn.Softmax()
        )
    def forward(self, x):
        return self.model(x)

In [ ]:
model = UserPredictionModel(X_train.shape[1])
loss_fn = nn.CrossEntropyLoss()
optimizer = th.optim.Adam(model.parameters(), lr=1e-4)

N_EPOCHS = 10

dataset_size = len(train_loader)

for epoch in range(N_EPOCHS):
    print(f"Epoch {epoch + 1}\n-------------------------------")

    for id_batch, (x_batch, y_batch) in enumerate(train_loader):
        y_batch = y_batch.to(device=device, dtype=th.int64)
        
        y_batch_pred = model(x_batch.float())
        
        loss = loss_fn(y_batch_pred, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if id_batch % 100 == 0:
            loss, current = loss.item(), (id_batch + 1)* len(x_batch)
            print(f"loss: {loss:>7f}  [{current:>5d}/{dataset_size:>5d}]")

Epoch 1
-------------------------------
loss: 1.398376  [   10/  922]
loss: 1.304929  [ 1010/  922]


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


loss: 0.988311  [ 2010/  922]
loss: 1.085007  [ 3010/  922]
loss: 1.123242  [ 4010/  922]
loss: 0.877060  [ 5010/  922]
loss: 0.871898  [ 6010/  922]
loss: 0.926782  [ 7010/  922]
loss: 0.870208  [ 8010/  922]
loss: 0.855819  [ 9010/  922]
Epoch 2
-------------------------------
loss: 0.855657  [   10/  922]
loss: 0.856602  [ 1010/  922]
loss: 0.842114  [ 2010/  922]
loss: 0.796413  [ 3010/  922]
loss: 0.767679  [ 4010/  922]
loss: 0.784216  [ 5010/  922]
loss: 0.786664  [ 6010/  922]
loss: 0.810088  [ 7010/  922]
loss: 0.879682  [ 8010/  922]
loss: 0.809463  [ 9010/  922]
Epoch 3
-------------------------------
loss: 0.779931  [   10/  922]
loss: 0.804448  [ 1010/  922]
loss: 0.826697  [ 2010/  922]
loss: 0.791040  [ 3010/  922]
loss: 0.772906  [ 4010/  922]
loss: 0.778199  [ 5010/  922]
loss: 0.802632  [ 6010/  922]
loss: 0.834349  [ 7010/  922]
loss: 0.845839  [ 8010/  922]
loss: 0.770098  [ 9010/  922]
Epoch 4
-------------------------------
loss: 0.758725  [   10/  922]
loss: 0.76

In [ ]:
running_accuracy = [] 
total = 0 
loss_fn = nn.CrossEntropyLoss()

for id_batch, (x_batch, y_batch) in enumerate(test_loader):
    y_batch = y_batch.to(device=device, dtype=th.int64)
    
    y_batch_pred = model(x_batch.float())
    
    loss = loss_fn(y_batch_pred, y_batch)
    y_pred = th.argmax(y_batch_pred, dim=1)
    running_accuracy.append(th.sum(y_pred == y_batch) / y_batch.size(0))
print('accuracy: ', sum(running_accuracy)/len(running_accuracy))


accuracy:  tensor(0.9935)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
